# GRB Afterglow Parameter Estimation with Redback

This notebook demonstrates how to perform Bayesian parameter estimation for GRB afterglows using VegasAfterglow models through the redback interface.

**Note:** VegasAfterglow is a pure physics library. All parameter estimation and MCMC fitting is done through [redback](https://github.com/nikhil-sarin/redback), which provides a comprehensive framework for transient analysis.

## Installation

```bash
pip install redback VegasAfterglow
```

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import redback
from redback.constants import day_to_s
import bilby

## 1. Load Data

Redback supports multiple ways to load GRB afterglow data:
- From Swift BAT+XRT catalogs
- From open access catalogs
- From custom data files

Here we demonstrate loading from custom CSV files.

In [ ]:
######### LOAD DATA FROM FILES #########

# Define the frequency bands [Hz] and corresponding light curve files
bands = [2.4e17, 4.84e14, 1.4e14]
lc_files = ["data/ep.csv", "data/r.csv", "data/vt-r.csv"]

# Load all data into lists
all_times = []
all_flux_densities = []
all_flux_density_errs = []
all_frequencies = []

for nu, fname in zip(bands, lc_files):
    df = pd.read_csv(fname)
    all_times.extend(df["t"].values)
    all_flux_densities.extend(df["Fv_obs"].values)
    all_flux_density_errs.extend(df["Fv_err"].values)
    all_frequencies.extend([nu] * len(df))

# Convert to numpy arrays
all_times = np.array(all_times)
all_flux_densities = np.array(all_flux_densities)
all_flux_density_errs = np.array(all_flux_density_errs)
all_frequencies = np.array(all_frequencies)

print(f"Loaded {len(all_times)} data points across {len(bands)} bands")
print(f"Time range: {all_times.min():.2e} - {all_times.max():.2e} s")
print(f"Frequency range: {all_frequencies.min():.2e} - {all_frequencies.max():.2e} Hz")

In [ ]:
# Create redback Afterglow object
afterglow = redback.transient.Afterglow(
    name='PowerLawJet_Wind',
    data_mode='flux_density',
    time=all_times,
    flux_density=all_flux_densities,
    flux_density_err=all_flux_density_errs,
    frequency=all_frequencies,
    redshift=1.58  # Set the redshift
)

print(f"Created afterglow object: {afterglow.name}")
print(f"Redshift: {afterglow.redshift}")
print(f"Luminosity distance: {afterglow.luminosity_distance:.3e} cm")

## 2. Plot the Data

Visualize the multi-band light curve data before fitting.

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))

colors = ['blue', 'orange', 'green']
shifts = [1, 1, 200]  # Visual shifts for clarity

for i, (nu, fname, color, shift) in enumerate(zip(bands, lc_files, colors, shifts)):
    df = pd.read_csv(fname)
    ax.errorbar(df["t"], df["Fv_obs"]*shift, df["Fv_err"]*shift, 
                fmt='o', markersize=5, label=f'{nu:.2e} Hz (×{shift})', 
                color=color, markeredgecolor='k', markeredgewidth=0.5, alpha=0.7)

ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlabel('Time [s]', fontsize=12)
ax.set_ylabel(r'$F_\nu$ [erg cm$^{-2}$ s$^{-1}$ Hz$^{-1}$]', fontsize=12)
ax.legend(fontsize=10)
ax.grid(alpha=0.3)
plt.title('Multi-band Afterglow Light Curves', fontsize=14)
plt.tight_layout()
plt.show()

## 3. Set Up Priors

Define priors for the power-law jet in wind medium model.

The VegasAfterglow model `vegas_powerlaw_wind` has the following parameters:
- `loge0`: log10 isotropic energy [erg]
- `g0`: Initial Lorentz factor
- `thc`: Core opening angle [rad]
- `thv`: Viewing angle [rad]
- `logepse`: log10 electron energy fraction
- `logepsb`: log10 magnetic field energy fraction
- `p`: Electron power-law index
- `logn0`: log10 wind parameter A* [g/cm]
- `k_e`: Energy power-law index (jet structure)
- `k_g`: Lorentz factor power-law index (jet structure)

In [ ]:
# Define priors using bilby
priors = bilby.core.prior.PriorDict()

# Energy and dynamics
priors['loge0'] = bilby.core.prior.Uniform(51, 54, 'loge0', latex_label=r'$\log_{10}(E_{\rm iso}$ [erg])')
priors['g0'] = bilby.core.prior.LogUniform(5, 1000, 'g0', latex_label=r'$\Gamma_0$')

# Jet geometry
priors['thc'] = bilby.core.prior.Uniform(0.01, 0.5, 'thc', latex_label=r'$\theta_c$ [rad]')
priors['thv'] = bilby.core.prior.Uniform(0.0, 0.1, 'thv', latex_label=r'$\theta_v$ [rad]')

# Microphysics
priors['p'] = bilby.core.prior.Uniform(2.0, 3.0, 'p', latex_label=r'$p$')
priors['logepse'] = bilby.core.prior.Uniform(-2, -0.5, 'logepse', latex_label=r'$\log_{10}(\epsilon_e)$')
priors['logepsb'] = bilby.core.prior.Uniform(-4, -0.5, 'logepsb', latex_label=r'$\log_{10}(\epsilon_B)$')

# Wind parameter
priors['logn0'] = bilby.core.prior.Uniform(-3, 1, 'logn0', latex_label=r'$\log_{10}(A_*)$ [g/cm]')

# Jet structure (fixed for this example)
priors['k_e'] = 2.0  # Fixed
priors['k_g'] = 2.0  # Fixed

# Fixed parameters (set by data)
priors['redshift'] = afterglow.redshift

print("Priors set up:")
for key in priors:
    print(f"  {key}: {priors[key]}")

## 4. Run Parameter Estimation

Use redback to fit the VegasAfterglow power-law jet model to the data.

**Note:** This cell will take some time to run. For a quick test, use `nlive=100`. For publication-quality results, use `nlive=2000` or higher.

In [ ]:
# Model-specific kwargs for VegasAfterglow
model_kwargs = {
    'frequency': afterglow.frequency,  # Pass frequency array for flux_density mode
    'output_format': 'flux_density'
}

# Run the fit
result = redback.fit_model(
    transient=afterglow,
    model='vegas_powerlaw_wind',  # VegasAfterglow power-law jet in wind
    sampler='dynesty',            # Nested sampling
    nlive=500,                    # Number of live points (increase for better results)
    priors=priors,
    model_kwargs=model_kwargs,
    outdir='redback_output',      # Output directory
    label='afterglow_powerlaw_wind',
    plot=True,                    # Generate plots automatically
    resume=True                   # Resume if interrupted
)

print("\nFitting complete!")
print(f"Output saved to: redback_output/")

## 5. Analyze Results

Print summary statistics and visualize the posterior distributions.

In [ ]:
# Print summary statistics
print("\n" + "="*60)
print("PARAMETER ESTIMATION RESULTS")
print("="*60)

# Get posterior samples
posterior = result.posterior

# Print median and 90% credible intervals for each parameter
for param in ['loge0', 'g0', 'thc', 'thv', 'p', 'logepse', 'logepsb', 'logn0']:
    if param in posterior:
        samples = posterior[param].values
        median = np.median(samples)
        lower = np.percentile(samples, 5)
        upper = np.percentile(samples, 95)
        print(f"{param:12s}: {median:.4f}  [{lower:.4f}, {upper:.4f}]")

print("\nLog Evidence (log Z):", result.log_evidence)
print("Log Evidence Error:", result.log_evidence_err)

## 6. Generate Corner Plot

Visualize parameter correlations and posterior distributions.

In [ ]:
# Generate corner plot
result.plot_corner()
plt.show()

## 7. Plot Light Curve with Best Fit

Compare the data to the model predictions using the maximum likelihood parameters.

In [ ]:
# Plot light curve with model
result.plot_lightcurve(random_models=100)  # Plot 100 random posterior samples
plt.show()

## 8. Custom Visualization

Create a custom multi-band plot showing data and best-fit model.

In [ ]:
# Get maximum likelihood parameters
max_likelihood_params = result.posterior.iloc[result.posterior['log_likelihood'].idxmax()]

# Generate model predictions
t_model = np.logspace(np.log10(all_times.min()), np.log10(all_times.max()), 200)

# Import the model function
from redback.model_library import all_models_dict
model_func = all_models_dict['vegas_powerlaw_wind']

# Plot
fig, ax = plt.subplots(figsize=(10, 7))

colors = ['blue', 'orange', 'green']
shifts = [1, 1, 200]

for i, (nu, fname, color, shift) in enumerate(zip(bands, lc_files, colors, shifts)):
    # Plot data
    df = pd.read_csv(fname)
    ax.errorbar(df["t"], df["Fv_obs"]*shift, df["Fv_err"]*shift, 
                fmt='o', markersize=6, label=f'{nu:.2e} Hz data (×{shift})', 
                color=color, markeredgecolor='k', markeredgewidth=0.5, alpha=0.7)
    
    # Plot model
    freq_array = np.full_like(t_model, nu)
    model_flux = model_func(t_model, redshift=afterglow.redshift, 
                           frequency=freq_array,
                           loge0=max_likelihood_params['loge0'],
                           g0=max_likelihood_params['g0'],
                           thc=max_likelihood_params['thc'],
                           thv=max_likelihood_params['thv'],
                           p=max_likelihood_params['p'],
                           logepse=max_likelihood_params['logepse'],
                           logepsb=max_likelihood_params['logepsb'],
                           logn0=max_likelihood_params['logn0'],
                           k_e=2.0, k_g=2.0,
                           output_format='flux_density')
    ax.plot(t_model, model_flux*shift, '-', color=color, linewidth=2, 
            label=f'{nu:.2e} Hz model (×{shift})', alpha=0.8)

ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlabel('Time [s]', fontsize=14)
ax.set_ylabel(r'$F_\nu$ [erg cm$^{-2}$ s$^{-1}$ Hz$^{-1}$]', fontsize=14)
ax.legend(fontsize=10, loc='best')
ax.grid(alpha=0.3)
plt.title('Best-Fit Power-Law Jet Model (Wind Medium)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('redback_output/bestfit_lightcurve.png', dpi=300, bbox_inches='tight')
plt.show()

print("Best-fit plot saved to: redback_output/bestfit_lightcurve.png")

## 9. Residuals Analysis

Examine the residuals to check model quality.

In [ ]:
result.plot_residuals()
plt.show()

## Next Steps

This notebook demonstrated basic parameter estimation with VegasAfterglow models through redback. For more advanced analysis:

**Try different models:**
- `vegas_tophat` - Uniform tophat jet + ISM
- `vegas_gaussian` - Gaussian jet + ISM  
- `vegas_two_component` - Two-component jet + ISM
- See [redback model library](https://redback.readthedocs.io/en/latest/models.html) for all available VegasAfterglow models

**Advanced features:**
- Enable reverse shock: add `reverse_shock=True` to `model_kwargs`
- Enable jet spreading: add `spread=True` to `model_kwargs`
- Load data from Swift: `redback.get_data.get_bat_xrt_afterglow_data_from_swift(grb='GRB070809')`
- Joint fitting of multiple transients
- Population inference

**Documentation:**
- [Redback documentation](https://redback.readthedocs.io/en/latest/)
- [VegasAfterglow documentation](https://yihanwangastro.github.io/VegasAfterglow/)
- [VegasAfterglow paper](https://ui.adsabs.harvard.edu/abs/2026JHEAp..5000490W/)